# v2 exploration trying to make it work better

In [ ]:
# Environment
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import sys
# Local modules
sys.path.append('..')
import reproducibility
import pandas as pd
from utils import print_dict_summary
from data_funcs import load_and_fix_data, rmse
from moisture_rnn import RNN, RNN_LSTM, create_rnn_data2
from moisture_rnn_pkl import pkl2train
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import mean_squared_error
from utils import hash2
import copy
import logging
import pickle
from utils import logging_setup
import yaml

In [ ]:
logging_setup()

## Test Data Creation

In [ ]:
train_file='train.pkl'
with open(train_file,'rb') as file:
    train=pickle.load(file)

## Test RNN

In [ ]:
with open("params.yaml") as file:
    params_all = yaml.safe_load(file)

params = params_all['rnn']

In [ ]:
import importlib
import moisture_rnn
importlib.reload(moisture_rnn)
from moisture_rnn import create_rnn_data2

In [ ]:
case = [*train.keys()][1]
print(case)

In [ ]:
params.update({'val_frac': .2, 'scale': True, 'scaler': 'standard'})
rnn_dat = create_rnn_data2(train[case], params)

In [ ]:
print(rnn_dat['X'].shape)
print(rnn_dat['features_list'])

In [ ]:
params.update({'features_list': ['wind', 'Ed', 'Ew', 'solar', 'lat', 'rain']})
params.update({'epochs': 200})

In [ ]:
print(rnn_dat['X'].shape)
print(rnn_dat['features_list'])

In [ ]:
reproducibility.set_seed()
rnn = RNN(params)
m, errs = rnn.run_model(rnn_dat)

In [ ]:
errs

## LSTM

In [ ]:
import importlib 
import moisture_rnn
importlib.reload(moisture_rnn)
from moisture_rnn import RNN_LSTM

In [ ]:
with open("params.yaml") as file:
    params = yaml.safe_load(file)["lstm"]
    
rnn_dat2 = create_rnn_data2(train[case],params)

In [ ]:
params.update({'epochs': 10})

In [ ]:
reproducibility.set_seed()
rnn = RNN(params)
m, errs = rnn.run_model(rnn_dat2)

In [ ]:
import importlib
importlib.reload(moisture_rnn)
from moisture_rnn import RNN_LSTM

In [ ]:
with open("params.yaml") as file:
    params = yaml.safe_load(file)["lstm"]

rnn_dat2 = create_rnn_data2(train[case],params)
params

In [ ]:
params.update({
    'learning_rate': 0.000001,
    'epochs': 10,
    'clipvalue':1.0
})

In [ ]:
reproducibility.set_seed()
lstm = RNN_LSTM(params)
m, errs = lstm.run_model(rnn_dat2)